In [1]:
import numpy as np
import cv2 
import mediapipe as mp
import tensorflow as tf
import keras.models
mp_draw=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose
model = keras.models.load_model('weight_pose.h5')

In [2]:
gpu=tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0],True)

RuntimeError: Physical devices cannot be modified after being initialized

In [ ]:
def detection(img,model):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img.flags.writeable=False
    results=model.process(img)
    img.flags.writeable=True
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    return img,results


In [ ]:
def draw_edit(landmarks,frame):
    h,w,_=frame.shape
    nose=np.array([[landmarks[mp_pose.PoseLandmark.NOSE].x*w,landmarks[mp_pose.PoseLandmark.NOSE].y*h]])
    lft_shoulder=np.array([[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x*w,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y*h]])
    rght_shoulder=np.array([[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x*w,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y*h]])
    neck=((lft_shoulder+rght_shoulder)/2+nose)/2
    chest_mid=(lft_shoulder+rght_shoulder)/2
    x1,y1=np.squeeze(neck)
    x2,y2=np.squeeze(chest_mid)
    cv2.circle(frame, (int(x1), int(y1)), 6, (0,255,0), -1)
    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,255,255), 4)
    

In [ ]:
def lay_mau(landmark,frame):
    a=[]
    for idx,lmk in enumerate(landmark):
        if idx in [x for x in range(1,11)]:
            continue
        elif idx in [x for x in range(17,23)]:
            continue
        y=landmark[0].y
        # cv2.putText(frame,'Cor:{}'.format(y),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1,cv2.LINE_AA)
        a.append(lmk.x)
        a.append(lmk.y)
        a.append(lmk.z)
    return a

In [ ]:
lst_sample=[]
frame_num=30
label=''
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5,model_complexity=1) as pose:
    cap=cv2.VideoCapture('C:/Users/PhuTuan/Downloads/VIDEO23_TEST.mp4')
    while cap.isOpened():
        ret,frame=cap.read()
        if ret:
            img,results=detection(frame,pose)
            mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
            try:
                landmark=results.pose_landmarks.landmark
                lst_sample.append(lay_mau(landmark,img))
                print(len(lst_sample))
                if len(lst_sample)==30:
                    tensor=np.expand_dims(lst_sample,axis=0)
                    lst_sample=[]
                    action=model.predict(tensor)
                    if action[0,0]>0.5:
                        label='1'
                    else:
                        label='0'
            except:
                pass
            cv2.putText(img,str(label),(50,50),cv2.FONT_HERSHEY_COMPLEX,2,(205,50,255),2,cv2.LINE_AA)
            if cv2.waitKey(20)&0xFF==ord('q'):
                break
            cv2.imshow('Pose',img)
        else:
            break
    cap.release()
cv2.destroyAllWindows()

In [ ]:
action

In [ ]:
# lst_sample=[]
# frame_num=30
# label=''
# with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5,model_complexity=1) as pose:
#     # for i in path:
#         cap=cv2.VideoCapture('chute07/cam5.avi')
#         while cap.isOpened():
#             ret,frame=cap.read()
#             if ret:
#                 img,results=detection(frame,pose)
#                 mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)

#                 landmarke=results.pose_landmarks.landmark
#                 landmark=results.pose_landmarks.landmark
#                 lst_sample.append(lay_mau(landmark,img))

#                 if len(lst_sample)==30:
#                     lst_sample_tensor=np.expand_dims(lst_sample,axis=0)[0]
#                     lst_sample=[]
#                     action=model.predict(lst_sample_tensor)
#                     if action[0,0]>0.5:
#                         label='1'
#                     else:
#                         label='0'
#                 # mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
#                 # lst_sample.append(samples)
#                 cv2.putText(img,str(label),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1,cv2.LINE_AA)
#                 if cv2.waitKey(20)& 0xFF==ord('q'):
#                     break
#                 # img=cv2.flip(img,1)
#                 cv2.imshow('Pose',img)
#             else:
#                 break
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
action